# Import libraries

In [ ]:
import sys
import os
import datetime as dt
from math import radians, cos, sin, asin, sqrt
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# %pip install sweetviz
import sweetviz as sv

from distutils.filelist import findall
import re

In [ ]:
def clean_pilot_name(df, name='Pilot', file_name='IGC File name'):
    """Cleans the pilot names in Lonestar 2022. Highly specific to these pilot names

    Args:
        df (DataFrame): data frame that contains the pilot names and file names
        name (str, optional): columns name of the pilot names. Defaults to 'Pilot'.
        file_name (str, optional): column name of the file names. Defaults to 'IGC File name'.
    """    

    df.loc[:, 'Pilot'] = df['IGC File name'].apply(
        lambda x: re.findall(r'[^0-9]+', x)[0].replace('Task', '').replace('-', '').replace(
            '_iljeg', 'Siljeg').replace('Davidsiljeg', 'David Siljeg').replace('Tom McCormick', 'Thomas McCormick'
                                        ).replace(
                                            'Corbin Lonestar', 'Corbin Petersen').replace(
                                                'Chris Dunlap', 'Christopher Dunlap'
                                                ).replace(
                                                    'Jaime Vargasaug', 'Jaime Vargas').strip().title())

    df.loc[:, 'Pilot'] = np.where(df['Pilot'].str.lower()=='esau', 'Esau Diaz', df['Pilot'])
    
    return(df)


# Import data

In [ ]:
## import data
ls_tracks_raw = pd.read_excel(r'Lonestar-2022-flights-analysis\data\raw\Lonestar2022.xlsx', header=0)
ls_tasks_raw = pd.read_excel(r'Lonestar-2022-flights-analysis\data\raw\Lonestar2022_results.xlsx', sheet_name=None)
print('ls_track shape: ', ls_tracks_raw.shape)
print('ls_tasks sheets: ', ls_tasks_raw.keys())


ls_track shape:  (216, 26)
ls_tasks sheets:  dict_keys(['Overall', 'Task1', 'Task2', 'Task3', 'Task4', 'Task5', 'Task6', 'Task7'])


# Clean data

## Tracks table

In [4]:
ls_tracks = clean_pilot_name(ls_tracks_raw, name='Pilot')


In [5]:
ls_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Row                216 non-null    int64  
 1   Flight Num         216 non-null    int64  
 2   Date               216 non-null    object 
 3   Pilot              216 non-null    object 
 4   Site               45 non-null     object 
 5   Launch Time        216 non-null    object 
 6   Flight Time        216 non-null    object 
 7   OLC Speed          216 non-null    float64
 8   Start Height       216 non-null    int64  
 9   Height Over Start  216 non-null    int64  
 10  Max Height         216 non-null    int64  
 11  Landing Height     216 non-null    int64  
 12  Height Gain        216 non-null    int64  
 13  Height Difference  216 non-null    int64  
 14  Min Vario          216 non-null    float64
 15  Max Vario          216 non-null    float64
 16  Max Gforce         216 non

In [6]:
# Convert the datetime columns to datetime type

ls_tracks['Task'] = pd.to_datetime(ls_tracks['Date'], dayfirst=True)
tasks_dict = dict(zip(sorted(ls_tracks['Task'].unique()),range(1, len(ls_tracks['Task'].unique())+1)))
ls_tracks['Task'] = ls_tracks['Task'].map(tasks_dict)

ls_tracks['Flight Duration (hr)'] = (pd.to_timedelta(ls_tracks['Flight Time'].astype(str)).dt.seconds/3600).round(1)

date_cols = ['Launch Time', 'Landing Time']
for col in date_cols:
    ls_tracks[col] = pd.to_datetime(ls_tracks[col].astype(str)).dt.time

ls_tracks.head()

,Row,Flight Num,Date,Pilot,Site,Launch Time,Flight Time,OLC Speed,Start Height,Height Over Start,...,OLC km,ID,Glider,GPS,Start Time (UTC),UTC Correction,Landing Time,IGC File name,Task,Flight Duration (hr)
0,1,1,07.08.2022,Max Montgomery,Hearne,11:52:27,05:19:07,35.23,89,2073,...,187.4,NaN,OZONE Zeno,Google Pixel 6 12 FW:0.9.7.6,16:52:27,-05:00:00,22:11:34,Max Montgomery 2022-08-07-XCT-MMO-01.igc,1,5.3
1,2,2,07.08.2022,Valter Castilho,NaN,12:01:42,01:44:15,0.00,87,1924,...,52.1,NKN,NKN,"Flymaster,LiveSD,B301",17:01:42,-05:00:00,18:45:57,Valter Castilho 2022-08-07 17_01_27.igc,1,1.7
2,3,3,07.08.2022,Felix Figueroa,Hearne,12:04:10,05:32:53,0.00,85,2055,...,184.6,NaN,GIN GLIDERS Puma,samsung SM-G973U 12 FW:0.9.8.1-beta,17:04:10,-05:00:00,22:37:03,Felix Figueroa 2022-08-07-XCT-FFI-01.igc,1,5.5
3,4,4,07.08.2022,Chris Lee,Hearne,12:04:43,05:32:18,0.00,83,2307,...,187.9,NaN,ADVANCE Omega XAlps 3,motorola moto g(7) power 10 FW:0.9.7.6,17:04:43,-05:00:00,22:37:01,Chris Lee 2022-08-07-XCT-CLE-01.igc,1,5.5
4,5,5,07.08.2022,Thomas Mccormick,NaN,12:09:18,05:33:50,0.00,87,2167,...,184.8,NKN,NKN,"Flymaster,DS",17:09:18,-05:00:00,22:43:08,Tom McCormick 2022-08-07_17_09_01.igc,1,5.6


In [9]:
ls_tasks_raw

{'Overall':     Number                Name               Glider GliderClass  Task 1   \
 0        9    Austin Kasserman         Ozone Zeno 2           D     1000   
 1       40         Tyler Burns       Ozone Mantra 7           D      846   
 2       23         Andrew Dahl         Ozone Zeno 2           D      911   
 3       20           Chris Lee  Advance Omega Xalps           D      715   
 4       35      David Prentice          777 Queen 2           C      510   
 5       49      Dustin Pachura          777 Queen 2           C      172   
 6       30      Max Montgomery           Ozone Zeno           D      787   
 7       18        Rick Fullmer         Gin Explorer           C      345   
 8       13          Ben Parker           Gin Camino           C      377   
 9       44      Felix Figueroa             Gin Puma           D      710   
 10      10       Moacir Thiele      777 Queen 2 Evo           C      345   
 11      45    Thomas Mccormick        Ozone Delta 4           C 

## Clean tasks table

In [38]:
tasks_list = [task for task in ls_tasks_raw.keys() if 'task' in task.lower()]
tasks_list

['Task1', 'Task2', 'Task3', 'Task4', 'Task5', 'Task6', 'Task7']

In [61]:
# Combine all the tasks in one table
overall = ls_tasks_raw['Overall']
overall.loc[:, 'Name'] = overall['Name'].str.title()
tasks_list = [task for task in ls_tasks_raw.keys() if 'task' in task.lower()]

for i in range(1,len(tasks_list)+1):
    global_var_name = 'task{}'.format(i)
    task = ls_tasks_raw['Task{}'.format(i)]
    task =  task.loc[task['Name'].isna()==False, :]
    task.loc[:, 'Name'] = task['Name'].str.title()
    task['Name'].replace('Davidsiljeg', 'David Siljeg', inplace=True)
    task['Task'] = i
    globals()[global_var_name] = task

tasks = pd.concat([globals()['task{}'.format(i)] for i in range(1,8)], ignore_index=True)
tasks

c:\Users\Gokcen\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\Users\Gokcen\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\Gokcen\AppData\Local\Temp/ipykernel_10472/3180808702.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,Id,Name,Gender,Nat,SS,ES,Time,km/h,Distance,Dist.,Time.1,Total,Task,Unnamed: 2,Altitude,Adj.
0,9.0,Austin Kasserman,M,USA,12:40:36,16:42:56,04:02:20,42.9,175.3,550.4,449.6,1000.0,1,NaN,NaN,NaN
1,23.0,Andrew Dahl,M,USA,13:10:54,17:23:50,04:12:56,41.1,175.3,550.4,360.7,911.0,1,NaN,NaN,NaN
2,27.0,David Siljeg,M,CRO,13:26:39,17:42:46,04:16:07,40.6,175.3,550.4,343.7,894.0,1,NaN,NaN,NaN
3,40.0,Tyler Burns,M,USA,12:54:06,17:20:35,04:26:29,39.0,175.3,550.4,295.7,846.0,1,NaN,NaN,NaN
4,21.0,Josh Mcveigh,M,USA,12:52:49,17:32:23,04:39:34,37.2,175.3,550.4,244.2,795.0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,29.0,John Davis,NaN,USA,NaN,NaN,NaN,NaN,7,37,NaN,37.0,7,M,NaN,NaN
233,14.0,Roxanne Lopez,NaN,USA,NaN,NaN,NaN,NaN,7,37,NaN,37.0,7,F,NaN,NaN
234,16.0,Ioan Miftode,NaN,USA,NaN,NaN,NaN,NaN,DNF,0,NaN,0.0,7,M,NaN,NaN
235,2.0,Jaime Vargas,NaN,COL,NaN,NaN,NaN,NaN,DNF,0,NaN,0.0,7,M,NaN,NaN


In [62]:
tasks['Distance'] = pd.to_numeric(tasks['Distance'].replace('DNF', 0))
tasks['GliderClass'] = tasks['Name'].map(dict(zip(overall['Name'], overall['GliderClass'])))

tasks.rename(columns={'Name': 'Pilot'}, inplace=True)
ls = pd.merge(tasks, ls_tracks, on=['Pilot', 'Task'])
ls['Made Goal'] = ls['Time'].isna()==False
tasks.shape

(237, 17)

In [65]:
ls.to_csv('lonestar_cleaned.csv', index=False)